# 0. Install and Import Dependencies

In [49]:
%pip install mediapipe opencv-python

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [50]:
!pip install nbimporter
!pip install nbformat


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [51]:
import cv2
import mediapipe as mp
import numpy as np
import time
import cv2
import numpy as np
import math
import os
from collections import deque
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

# 1. Swipe Detection

In [52]:
def leftSwipeDetection(leftWristLocations):
    return leftWristLocations[0][0] - leftWristLocations[-1][0] > 250

def rightSwipeDetection(rightWristLocations):
    return rightWristLocations[-1][0] - rightWristLocations[0][0] > 250

def upSwipeDetection(rightWristLocations):
    return rightWristLocations[0][1] - rightWristLocations[-1][1] > 250


In [ ]:

def get_shirt_ratio(pose_points, img):
    width = int(math.fabs(pose_points[1][0] - pose_points[0][0]))
    aspect_ratio = img.shape[0] / img.shape[1]
    height = int(width * aspect_ratio)
    shirt_ratio = width / height if height != 0 else 0
    return shirt_ratio

def rotation_angle(pose_points):
    dx = pose_points[1][0] - pose_points[0][0]
    dy = pose_points[1][1] - pose_points[0][1]
    return math.degrees(math.atan2(dy, dx))

def mapClothes(pose_points, img):
    ratio = get_shirt_ratio(pose_points, img)
    fixedRatio = 262 / 190
    heightToWidth = img.shape[0] / img.shape[1]
    width = int(math.fabs(pose_points[1][0] - pose_points[0][0]) * fixedRatio)
    height = int(width * heightToWidth)
    x_offset = 40
    src = np.array([[0, 0], [width, 0], [width, height], [0, height]], dtype='float32')
    dest = np.array([
        [pose_points[0][0] , pose_points[0][1]],
        [pose_points[1][0] , pose_points[1][1]],
        [pose_points[2][0] - x_offset, pose_points[2][1]],
        [pose_points[3][0] + x_offset, pose_points[3][1]]
    ], dtype='float32')
    transform = cv2.getPerspectiveTransform(src, dest)
    angle = rotation_angle(pose_points)
    center = img.shape[1] // 2, img.shape[0] // 2
    scale = math.fabs(pose_points[1][0] - pose_points[0][0]) / 190
    mat = cv2.getRotationMatrix2D(center, -angle, 1.0)
    img = cv2.resize(img, (width, height))
    offset = [int(44 * scale), int(48 * scale)]
    return img, transform


def capture_and_rotate_frame(camera, angle):
    ret, frame = camera.read()
    if not ret:
        return None

    # Rotate the frame according to the specified angle
    if angle == 90:
        frame = cv2.rotate(frame, cv2.ROTATE_90_CLOCKWISE)
    elif angle == -90:
        frame = cv2.rotate(frame, cv2.ROTATE_90_COUNTERCLOCKWISE)
    elif angle == 180:
        frame = cv2.rotate(frame, cv2.ROTATE_180)

    return frame

In [55]:
leftWristLocations = deque(maxlen=10)
rightWristLocations = deque(maxlen=10)


In [56]:
cap = cv2.VideoCapture(0)
angle = 0

last_left_swipe_time = 0
last_right_swipe_time = 0
swipe_delay = 1

clothing_tops = ["White", "Blue", "Red"]
clothing_bottoms = ["Skirt", "Slacks", "Skorts", "Cargo pants", "Low-Rise Jeans"]
clothing_tops_index = 0
clothing_bottoms_index = 0
clothing_top_selected = False

## Setup mediapipe instanceq
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while True:
        print("Starting frame capture and rotation")
        frame = capture_and_rotate_frame(cap, angle)
        if frame is None: 
            print("Frame capture returned None; ending loop")
            break


        #ret, frame = cap.read()
        
        listTops = os.listdir("images")
        
        # print(f"OG Shirt Image Shape: {img.shape}")
        #img = cv2.resize(img, (100,100))
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates
            
            left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
            
            left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]

            left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]

            left_shoulder_coords = tuple(np.multiply(left_shoulder, [640, 480]).astype(int)) ## Normalizes Coordinates
            right_shoulder_coords = tuple(np.multiply(right_shoulder, [640, 480]).astype(int)) ## Normalizes Coordinates

            left_hip_coords = tuple(np.multiply(left_hip, [640, 480]).astype(int)) ## Normalizes Coordinates
            right_hip_coords = tuple(np.multiply(right_hip, [640, 480]).astype(int)) ## Normalizes Coordinates

            upper_coords = [left_shoulder_coords, right_shoulder_coords, right_hip_coords, left_hip_coords]

            img = cv2.imread(os.path.join("images", listTops[clothing_tops_index]),cv2.IMREAD_UNCHANGED)

            img, transform = mapClothes(upper_coords, img)

            img = cv2.warpPerspective(img, transform, (frame.shape[1], frame.shape[0]))

            print("Mapped and transformed clothing image")

            cv2.imshow("Transformed shirt", img)
            #cv2.addWeighted(new_img, 0.5, frame, 0.5, 0, frame)


        

            
            
            #add elbows, neck, chest, and waist later
            left_wrist_coords = tuple(np.multiply(left_wrist, [640, 480]).astype(int)) ## Normalizes Coordinates
            right_wrist_coords = tuple(np.multiply(right_wrist, [640, 480]).astype(int)) ## Normalizes Coordinates

            leftWristLocations.append(left_wrist_coords)
            rightWristLocations.append(right_wrist_coords)
            
            
            current_time = time.time()

            if leftSwipeDetection(leftWristLocations):
                if current_time - last_left_swipe_time > swipe_delay:
                    print("Swiped Right: index: ", clothing_tops_index)
                    last_left_swipe_time = current_time
                    if clothing_tops_index == 0:
                        clothing_tops_index = len(clothing_tops) - 1
                        
                    else:
                        clothing_tops_index -= 1
                        
                    '''if imageNum==0:
                        imageNum = len(listTops) -1
                        
                    else: 
                        imageNum -=1
                        print("decremented",imageNum)'''
                
            if rightSwipeDetection(rightWristLocations):
                if current_time - last_right_swipe_time > swipe_delay:
                    print("Swiped Left: index: ", clothing_tops_index)
                    last_right_swipe_time = current_time
                    if clothing_tops_index == len(clothing_tops) - 1 :
                        clothing_tops_index = 0
                        
                    else:
                        clothing_tops_index += 1
                        
                    '''if imageNum== len(listTops)-1:
                        imageNum = 0
                    else: 
                        imageNum +=1
                        print("incremented",imageNum)'''
           
            if upSwipeDetection(rightWristLocations):
                if current_time - last_right_swipe_time > swipe_delay:
                    print("Swiped Up: index: ", clothing_tops_index)
                    last_right_swipe_time = current_time
                    clothing_top_selected = not clothing_top_selected
                    if clothing_top_selected: #if the boolean is false that means bottom array selected else top array
                        print("top array")
                    else:
                        print("bottom array")
            '''img = cv2.imread(os.path.join("images", listTops[imageNum]),cv2.IMREAD_UNCHANGED)

            img, transform = mapClothes(upper_coords, img)

            img = cv2.warpPerspective(img, transform, (frame.shape[1], frame.shape[0]))

            print("Mapped and transformed clothing image")'''
             #test for shoulders
            cv2.putText(image, f"Left shoulder: ({left_shoulder_coords[0]}, {left_shoulder_coords[1]})", 
                        left_shoulder_coords, 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            cv2.putText(image, f"Right shoulder: ({right_shoulder_coords[0]}, {right_shoulder_coords[1]})", 
                        right_shoulder_coords, 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            cv2.putText(image, f"Right hip: ({right_hip_coords[0]}, {right_hip_coords[1]})", 
                        right_hip_coords, 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            cv2.putText(image, f"Left hip: ({left_hip_coords[0]}, {left_hip_coords[1]})", 
                        left_hip_coords, 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            

            #Display wrist coordinates at the wrist location
            cv2.putText(image, f"Left Wrist: ({left_wrist_coords[0]}, {left_wrist_coords[1]})", 
                        left_wrist_coords, 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Display wrist coordinates at the wrist location
            cv2.putText(image, f"Right Wrist: ({right_wrist_coords[0]}, {right_wrist_coords[1]})", 
                        right_wrist_coords, 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
        except Exception as e:
            print(f"An error occurred during pose estimation or clothing mapping: {e}")

        except:
            pass
            
       
        # Draw a rectangle at the top of the screen for clothing item display
        cv2.rectangle(image, (0, 0), (225, 73), (245, 117, 16), -1)

        # Display clothing item name
        cv2.putText(image, 'CLOTHING ITEM', (15, 12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
        if(clothing_top_selected):
            cv2.putText(image, clothing_tops[clothing_tops_index], 
                        (10, 60), 
                        cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)
        else:
             cv2.putText(image, clothing_bottoms[clothing_tops_index], 
                        (10, 60), 
                        cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)
        
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )               
        
        cv2.imshow('Rotated Frame', image)
        #cv2.imshow('Transformed Shirt', img)
        #cv2.imshow('Warped Image', new_img)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

Starting frame capture and rotation


c:\Users\15rak\Documents\MagicMirror\env\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


Mapped and transformed clothing image
Starting frame capture and rotation
Mapped and transformed clothing image
Starting frame capture and rotation
Mapped and transformed clothing image
Starting frame capture and rotation
Mapped and transformed clothing image
Starting frame capture and rotation
Mapped and transformed clothing image
Starting frame capture and rotation
Mapped and transformed clothing image
Starting frame capture and rotation
Mapped and transformed clothing image
Starting frame capture and rotation
Mapped and transformed clothing image
Starting frame capture and rotation
Mapped and transformed clothing image
Starting frame capture and rotation
Mapped and transformed clothing image
Starting frame capture and rotation
Mapped and transformed clothing image
Starting frame capture and rotation
Mapped and transformed clothing image
Starting frame capture and rotation
Mapped and transformed clothing image
Starting frame capture and rotation
Mapped and transformed clothing image
